# DiCoco

Florian Clochepin

In [14]:
# import des différentes librairies
from IPython.display import display, clear_output, HTML, Markdown 
import ipywidgets as widgets
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import numpy as np
from moviepy.editor import AudioFileClip
from pylab import specgram, show
%run florian_clochepin_scrapping.ipynb import getWord

In [22]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            0.12.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backports.weakref                  1.0

In [2]:
df = pd.read_csv("../data/dataframe/BuchananRich.csv" , delimiter=";")

In [3]:
#Création des boutons de filtre
filterHwd = widgets.Text(
       description='Word : ',
       layout={'width': '200px'} )
filterIPA = widgets.Text(
       description='IPA : ',
       layout={'width': '200px'} )
slider = widgets.IntSlider(value=3,min=1,max=10,layout={'width': '200px'}) 
bouton = widgets.Button(description="Collecter")

In [4]:
def transformHTML(row):
    content = "<tr>"
    content += "<td>" + row["motVedette"] + "</td>"
    content += "<td>" + str(row["nbSyll"]) + "</td>"
    content += "<td>" + row["POS"] + "</td>"
    content += "<td>" + row["pron"] + "</td>"
    content += "<td>" + row["sampa"] + "</td>"
    content += "<td>" + row["ipa"] + "</td>"
    content += "</tr>"
    return content

In [5]:
def getMP3(row):
    listReturn = []
    tabCorrespondance = getWord(row["motVedette"])
    #On vérifie qu'une correspondance est trouvée
    if(len(tabCorrespondance)>0):
        #Si le tableau retourné n'est pas vide, on conserve la part of speech
        for ind in df.index:
            if tabCorrespondance["Pos"][ind] == row["POS"]:
                #print(tabCorrespondance["Sound"][ind])
                if "anglais américain" in tabCorrespondance["Sound"][ind].keys():
                    listReturn.append(tabCorrespondance["Sound"][ind]["anglais américain"])
                else:
                    listReturn.append("Fichier manquant")
                if "anglais britannique" in tabCorrespondance["Sound"][ind].keys():
                    listReturn.append(tabCorrespondance["Sound"][ind]["anglais britannique"])
                else:
                    listReturn.append("Fichier manquant")
                return listReturn
            else:
                return ["Fichier manquant", "Fichier manquant"]
    else:
        return ["Fichier manquant", "Fichier manquant"]    

In [19]:
def transformMP3(row):
    content = "<li><h3>" + row["motVedette"] + "</h3>"
    content += row["pron"]
    content += "<br><b>Anglais britannique : </b>"
    if row["mp3"][1] == "Fichier manquant":
        content += "Fichier manquant      "
    else:
        content += "<audio controls src='" + row["mp3"][1] + "'></audio>"
    content += "<b>Anglais américain : </b>"
    if row["mp3"][0] == "Fichier manquant":
        content += "Fichier manquant"
    else:
        content += "<audio controls src='" + row["mp3"][0] + "'></audio>"
    content += "</li><br><br>"
    return content

In [20]:
out1 = widgets.Output()
out2 = widgets.Output()

def on_button_clicked(_):
    filteredDF = df.copy()
    if filterHwd.value != "":
        filteredDF = df[df["word"].str.contains(filterHwd.value,regex=True)]
    if filterIPA.value != "" and filteredDF.shape[0] >= 1:
        filteredDF = filteredDF[filteredDF["ipa"].str.contains(filterIPA.value,regex=True)]
    if filteredDF.shape[0] >= 1:
        filteredDF = filteredDF.head(slider.value)
        filteredDF["html"] = filteredDF.apply(lambda x: transformHTML(x),1)
        filteredDF["mp3"] = filteredDF.apply(lambda x: getMP3(x), 1)
        filteredDF["htmlmp3"] = filteredDF.apply(lambda x: transformMP3(x), 1)
        htmlContent = "<br><br><table><thead><tr><th>Mot</th><th>Nombre de syllabes</th><th>POS</th><th>Prononciation originelle</th><th>Prononciation Sampa</th><th>Prononciation IPA</th></tr></thead>"
        with out1:
            clear_output()
            display(HTML(htmlContent + '<tbody>' + "".join(list(filteredDF['html'])) + '</tbody></table>'))
        with out2:
            clear_output()
            display(HTML("<br><p>Attention, si des fichiers MP3 sont affichés comme manquants, il est probable que Linguee ait bloqué la requête.</p>"))
            display(HTML('<ul>' + "".join(list(filteredDF['htmlmp3'])) + '</ul>'))
    else:
        with out1:
            clear_output()
            display(HTML("<br><p>Aucune correspondance n'a été trouvée</p>"))
        with out2:
            clear_output()
bouton.on_click(on_button_clicked)

In [21]:
vbFiltres = widgets.HBox([filterHwd, filterIPA, slider, bouton])
vbOuts = widgets.VBox([out1, out2])
hbAllWidgets = widgets.VBox([vbFiltres, vbOuts])
hbAllWidgets